### Assignment of Gender to the Syntehtic Population

Beware ye that searced for 'Gender Assignment' that ye shall not find waht you are looking for. Here I play God for my synthetic population. Now, let's get on with the scheduled program. (I will not be apologising for that pun.)

Now, this notebook covers the code, choices and logic behind assigning gender to a synthetic population of agents for the city of Den Haag. The source data for this notebook is from https://denhaag.incijfers.nl/viewer

We begin by importing the required libraries for this process.

In [15]:
import numpy as np
import pandas as pd
np.random.seed(42)
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys


Now we import the raw data file that has been downloaded as an Excel file from Den Haag in Cijfers website. This dataset focuses on only the 2 gender division. Since this is how the source data chooses to classify gender, we will adhere to this system. 

While simpler and easier for me to code, it is important to expect edge cases and prepare for their inclusion. I won't be doing it in this case, but needs to be noted in future expansions. 

In [16]:
df_counts = pd.read_excel("Demographics/gender_raw_data.xlsx", header=0)
df_counts.columns = df_counts.iloc[0]  # Set first row as header
df_counts = df_counts.iloc[1:].reset_index(drop=True)  # Remove the first row from data
df_counts = df_counts.rename(columns={df_counts.columns[0]: "Neighbourhood"})
df_counts.head()

,Neighbourhood,Number of residents per 1-1,Gender: male,Gender: female
0,Oud Scheveningen,2925,1455,1470
1,Vissershaven,4894,2350,2544
2,Scheveningen Badplaats,5656,2768,2888
3,Visserijbuurt,4107,1928,2179
4,v Stolkpark/Schev Bosjes,795,384,411


In [17]:
num_agents =  df_counts["Number of residents per 1-1"].sum()
print(num_agents)
num_agents =  df_counts["Gender: male"].sum()
print(num_agents)
num_agents =  df_counts["Gender: female"].sum()
print(num_agents)
num_agents = df_counts["Gender: male"].astype(float).sum() + df_counts["Gender: female"].astype(float).sum()
print(num_agents)

569387
283959
285428
569387.0


Fortunately this time, we have every single person being accounted for. However a quick check for NaN values is a must

In [18]:
# Check for NaN values in all columns
print(df_counts.isna().sum())

0
Neighbourhood                  0
Number of residents per 1-1    3
Gender: male                   3
Gender: female                 3
dtype: int64


Okay, since we have no missing people we can safely set these values to 0 and move on with this section. 

In [19]:
df_counts = df_counts.fillna(0)
print(df_counts.isna().sum())

0
Neighbourhood                  0
Number of residents per 1-1    0
Gender: male                   0
Gender: female                 0
dtype: int64


C:\Users\nachi\AppData\Local\Temp\ipykernel_20748\3913859920.py:1: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_counts = df_counts.fillna(0)


Now that the data has been cleaned, we can move towards the step for assigning gender to the agents. 

In [20]:
import pandas as pd
import numpy as np
import random

# Set the random seed for reproducibility
random.seed(42)

# Read in the agents file
agents = pd.read_csv('agents.csv')

# Read in the gender distribution data (make sure the file path and column names match your data)
gender_df = df_counts

# Prepare to assign gender
genders = []

for _, agent in agents.iterrows():
    neighbourhood = agent['neighbourhood']
    # Get the row in gender_df corresponding to the agent's neighbourhood
    neighbourhood_row = gender_df[gender_df['Neighbourhood'] == neighbourhood]
    if neighbourhood_row.empty:
        # If neighbourhood not found, assign NaN or skip
        genders.append(np.nan)
        continue

    # Get counts for male and female
    try:
        male_count = float(neighbourhood_row['Gender: male'].values[0])
    except (KeyError, IndexError, ValueError):
        male_count = 0
    try:
        female_count = float(neighbourhood_row['Gender: female'].values[0])
    except (KeyError, IndexError, ValueError):
        female_count = 0

    total = male_count + female_count
    if total == 0:
        genders.append(np.nan)
        continue

    # Assign gender based on the neighbourhood's composition
    gender = random.choices(
        population=['male', 'female'],
        weights=[male_count, female_count],
        k=1
    )[0]
    genders.append(gender)

# Add the "Gender" column to the agents DataFrame
agents['Gender'] = genders

# Check how many NaNs were assigned
print("Number of NaN genders assigned:", agents['Gender'].isna().sum())

# Preview the result
agents.head()

Number of NaN genders assigned: 0


,Individual_ID,neighbourhood,Age,Gender
0,447607,Oud Scheveningen,70,female
1,515489,Oud Scheveningen,2,male
2,133483,Oud Scheveningen,31,male
3,130686,Oud Scheveningen,20,male
4,499841,Oud Scheveningen,79,female


In [21]:
df_counts.to_csv('Demographics/gender_cleaned.csv', index=False)
agents.to_csv('agents.csv', index=False)

And that concludes a rather anticlimactic step in this process